# Longitudinal Distance

In [ ]:
from pathlib import Path
# from project_heart.lv import LV
from project_heart.utils import set_jupyter_backend
from project_heart.enums import *
set_jupyter_backend("pythreejs")
import logging
import numpy as np
float_formatter = "{:.5f}".format
np.set_printoptions(formatter={'float_kind':float_formatter})

from project_heart.examples import get_lv_ideal
from project_heart.examples import get_lv_typeA

lv_ideal = get_lv_ideal(Path("../../_static/sample_files/ideal_linear_pressure_increase copy.xplt"))
lv_typeA = get_lv_typeA(
    Path("../../_static/sample_files/lv_typeA_hex.vtk"),
    Path("../../_static/sample_files/sample_displacement_lv_typeA_hex_states_linear_press_incr.pbz2")
    )

sample_spk_typeA_endo = lv_typeA.get_speckles(spk_name="SAMPLE", spk_group="endo", spk_collection="SAMPLE")[0]
sample_spk_typeA_epi = lv_typeA.get_speckles(spk_name="SAMPLE", spk_group="epi", spk_collection="SAMPLE")[0]

## Longitudinal Distance

**Definition**: Perpendicular distance between reference apex and reference base.

The longitudinal distance is defined as the shortest distance between a reference node at base region and a reference node at apex region. In our implementation, we apply this computation to one or more nodesets and reduce the final value, allowing us to compute the longitudinal distance at Endocardium and Epicardium and record the 'mean' longitudinal distance.

For each nodeset and each timestep, we extract the top-most and lower-most nodes and compute the centroid for base and apex, respecitively. The final value for each timestep is used in the reduction method.

Here is a visual representation *(note that longitudinal lines for 'extreme' approach are in an offeset from longitudinal axis for visualization purposes)*:

In [ ]:
ld = lv_ideal.plot_longitudinal_distance(
    nodesets = {lv_ideal.REGIONS.ENDO, lv_ideal.REGIONS.EPI},
    approach="extremes",
)

In [ ]:
lds = lv_ideal.longitudinal_distances(
    recompute=True,
    nodesets = {lv_ideal.REGIONS.ENDO, lv_ideal.REGIONS.EPI},
    approach="extremes",
    t=0.0,
    log_level=logging.ERROR, 
)

In [ ]:
print("Sample longitudinal distance:", ld)
print("Reduced longitudinal distance by internal algorithm:", lds)

### 'Extremes' vs 'Estimate apex and base'

Although the computation for longitudinal distance seems straightforward, there is reseonable doubt reggarding how to choose top-most and bottom-most reference nodes. In our implementation, we have two approaches: ***extremes*** and ***estimate_apex_base***. 

The first method chooses the apex and base nodes based on minimum and maximum, respectively, along the Z axis; moreover, because there might not exist a node at specified location, we simply compute the distance along the found z values. Therefore, this approach is very constrained on the assumption that the geometry's longitudinal axis is closely aligned with the Z axis.

On the other hand, the second approach applies the 'est_apex_and_base_refs_iteratively' method, which finds the approximate location of apex and base locations. This method returns a reference node for Apex and Base for that given nodeset, which allows for the euclidean distance between the two points and, since the estimation methods works iteractively (see docs for details), this methods does not imposes a strict aligment with the Z axis. 

Here is a visualization with both approaches *(note that longitudinal lines for 'extreme' approach are in an offeset from longitudinal axis for visualization purposes)*:

In [ ]:
ld_ex = lv_typeA.plot_longitudinal_distance(
    nodesets = {lv_typeA.REGIONS.ENDO, lv_typeA.REGIONS.EPI},
    approach="extremes",
    t=0.15
)

In [ ]:
ld_es = lv_typeA.plot_longitudinal_distance(
    nodesets = {lv_typeA.REGIONS.ENDO, lv_typeA.REGIONS.EPI},
    approach="estimate_apex_base",
    t=0.15
)

In [ ]:
print("with 'extremes:", ld_ex)
print("with 'estimate_apex_base':", ld_es)

Considering that non-ideal geometries have irregular shapes at apex and base regions for all nodesets, we recommend using 'estimate_apex_base', which automatically finds the best estimation for reference nodes at each nodeset.

## TODO: Using longitudinal distance